In [64]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np
import configparser
from datetime import date
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from datetime import date
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.feature_selection import chi2, f_regression, SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import iqr
from sklearn.preprocessing import scale, PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import yfinance as yf
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC


import warnings
warnings.simplefilter(action='ignore', category=(FutureWarning))


pd.set_option('use_inf_as_na',True)
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [65]:
comission = {}
with open('/Users/p.matchenkov/Desktop/comissions.txt', 'r') as f:
    for line in f:
        (key, val) = line.split(':')
        comission[key] = float(str(val)[:-1])/100
comission_df = pd.DataFrame(list(comission.items()), columns=['order_symbol', 'comission_rate'])


def connect():
    config = configparser.ConfigParser()
    config.read('/Users/p.matchenkov/NotebookProjects/configurations/config.ini')
    password = config['PASSWORDS']['password']
    localhost = config['LOCALHOST']['localhost']
    db_name = config['NAMES']['bd_name']
    db_type = config['NAMES']['bd_type']
    login = config['PASSWORDS']['login']

    engine = create_engine(f'{db_type}://{login}:{password}@{localhost}/{db_name}')
    try:
        engine.connect()
        #print('connections success')
    except Exception as err:
        print(err)
    return engine

def connect_to_cexlive():
    config = configparser.ConfigParser()
    config.read('/Users/p.matchenkov/NotebookProjects/configurations/config.ini')
    password = config['PASSWORDS']['cexlive_password']
    localhost = config['CEX_prod']['localhost']
    db_name = config['CEX_prod']['bd_name']
    db_type = config['CEX_prod']['bd_type']
    login = config['CEX_prod']['login']

    engine = create_engine(f'{db_type}://{login}:{password}@{localhost}/{db_name}')
    conn = engine.connect()
    try:
        engine.connect()
        #print('connections success')
    except Exception as err:
        print(err)
    return engine

def get_price_from_db(dates):
    engine = connect()
    request = f"""
    select symbol, close, date_added 
    from market_data
    where symbol in ('BTC/USD', 'EUR/USD', 'GBP/USD', 'ETH/USD') and date_added in {dates} and candle = 'h'
    """
    df = pd.DataFrame(engine.execute(request)).drop_duplicates(subset=['symbol', 'date_added'])\
        .rename(columns={'symbol': 'order_symbol', 'date_added': 'executed_date'})
    df['quote_currency'] = df['order_symbol'].str.extract('/(.*)')
    req_jpy = f"""
    select symbol, (1/`close`), date_added 
    from market_data
    where symbol = 'USD/JPY' and date_added in {dates} and candle = 'h'
    """
    jpy_df = pd.DataFrame(engine.execute(req_jpy)).drop_duplicates(subset=['symbol', 'date_added'])\
        .rename(columns={'divide(1, close)': 'close', 'symbol': 'order_symbol', 'date_added': 'executed_date'})
    jpy_df['quote_currency'] = jpy_df['order_symbol'].str.extract('(.*)/')
    df = pd.concat([df, jpy_df]).drop(columns='order_symbol')
    return df

def get_deposit(users):
    engine = connect()
    request = f"""
    select * 
    from finance_statistics_hour 
    where record_type in ('deposit', 'withdrawal')
    and symbol != 'EUR' 
    and user_id in {tuple(users)}
    order by date_time_added
    """
    deps = pd.DataFrame(engine.execute(request)).drop_duplicates()[['user_id', 'account_id', 'symbol', 'record_type', 'total_amount', 'date_added']]\
        .rename(columns={'date_added': 'executed_date', 'symbol': 'quote_currency'})
    deps = deps.merge(get_price_from_db(tuple(deps['executed_date'].astype(str).unique())),
                      how='left', on=['executed_date', 'quote_currency'])\
                      .fillna(1)
    deps['deposit'] = deps['total_amount'].loc[deps['record_type'] == 'deposit'] * deps['close']
    deps['withdrawal'] = deps['total_amount'].loc[deps['record_type'] == 'withdrawal'] * deps['close']
    deps['date'] = pd.to_datetime(deps['executed_date']) #- pd.to_timedelta(7, unit='d')
    deps = deps.groupby('user_id').agg({'deposit': ['sum', 'mean'], 'withdrawal': ['sum', 'mean']}).reset_index()
    deps.columns = ['user_id', 'deposit', 'mean_deposit', 'withdrawal', 'mean_withdrawal']
    return deps

def avg_trade_time(orders):
    df = orders[['user_id', 'account_id', 'executed_date_time', 'position_code']].loc[orders['position_effect'] == 'OPENING']\
        .merge(orders[['executed_date_time', 'position_code']].loc[orders['position_effect'] == 'CLOSING'], how='left', on = 'position_code')
    
    df['time_open'] = pd.to_datetime(df['executed_date_time_x']) #- pd.to_timedelta(7, unit='d')
    df['time_close'] = pd.to_datetime(df['executed_date_time_y']) #- pd.to_timedelta(7, unit='d')

    df['trade_time'] = (df['time_close'] - df['time_open'])#.total_seconds()
    df['trade_time'] = df['trade_time'] / np.timedelta64(1, 'h')
    df = df.drop(columns={'executed_date_time_x', 'position_code', 'executed_date_time_y'})
    
    df = df.groupby('user_id').agg({'trade_time': 'mean'}).reset_index()\
        .rename(columns={'time_close': 'date'})
    return df

def get_account_leverage(accounts):
    req = f"""
    select name, a.account_code from account_groups ag 
    join account_to_groups atg on ag.id = atg.group_id join accounts a on a.id = atg.account_id 
    where name like ('Leverage%%') and a.clearing_code = 'live'
    and a.account_code in {tuple(accounts)}
    """
    engine = connect_to_cexlive()
    df = pd.DataFrame(engine.execute(req)).drop_duplicates().rename(columns={'account_code': 'account_id'})
    return df

def get_lag_btw_dep_trade(users):
    req = f"""
    select min(executed_date_time), user_id, account_created_date_time 
    from order_statistics os
    where user_id in {tuple(users)}
    group by user_id, account_created_date_time
    """
    engine = connect()
    df = pd.DataFrame(engine.execute(req))
    df.columns = ['executed_date_time', 'user_id', 'created_date_time']
    df = df.sort_values('executed_date_time', ascending=True).drop_duplicates('user_id')
    df['first_time_trade'] = (df['executed_date_time'] - df['created_date_time']).dt.days
    df = df.drop(columns={'executed_date_time', 'created_date_time'})
    return df

def get_first_deposit(users):
    engine = connect()
    request = f"""
    select  min(date_added) as executed_date , user_id, symbol as quote_currency, total_amount
        from finance_statistics_hour 
        where record_type = 'deposit' and user_id in {tuple(users)}
    GROUP by  user_id, symbol, total_amount
        order by executed_date
    """
    first_deps = pd.DataFrame(engine.execute(request)).drop_duplicates('user_id')[['user_id','quote_currency', 'total_amount', 'executed_date']]
        
    dates = tuple(first_deps['executed_date'].astype(str).unique())
    first_deps = first_deps.merge(get_price_from_db(dates), how='left', on=['executed_date', 'quote_currency'])
    first_deps.loc[first_deps['quote_currency']=='USDT', 'close'] = first_deps['close'].loc[first_deps['quote_currency']=='USDT'].fillna(1)
    first_deps['first_dep_usd'] = first_deps['total_amount'] * first_deps['close']
    return first_deps[['user_id', 'first_dep_usd']]

def get_btc_price(dates):
    engine = connect()
    request = f"""
    select symbol, close as btc, date_added as created_date
    from market_data
    where symbol in ('BTC/USD') and date_added in {tuple(dates)} and candle = 'd'
    group by symbol, btc, created_date
    """
    df = pd.DataFrame(engine.execute(request)).drop_duplicates(subset=['created_date'])
    df['created_date'] = df['created_date'].astype(str)
    df['symbol'] = df['symbol'].str.extract('/(.*)')
    return df

In [66]:
def accuracy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return accuracy_score(y_true, y_pred)

def precision(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    return precision_score(y_true, y_pred)

def recall(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    return recall_score(y_true, y_pred)


In [67]:
def get_metrics(y_train: np.ndarray,
                y_train_pred: np.ndarray,
            
                y_test: np.ndarray,
                y_pred: np.ndarray,
                name: str,
                model,
                x: np.ndarray,
                y: np.ndarray,
                ):  
    """Генерация таблицы с метриками"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]

    df_metrics['train_accuracy'] = accuracy_score(y_train, y_train_pred)
    df_metrics['test_accuracy'] = accuracy_score(y_test, y_pred)

    df_metrics['recall'] = recall_score(y_test, y_pred)
    df_metrics['precision'] = precision_score(y_test, y_pred)

    df_metrics['F1'] = f1_score(y_test, y_pred)

    df_metrics['cv_roc_auc'] = cross_val_score(model, x, y, cv=5, scoring='roc_auc').mean() #verbose=2
    df_metrics['pr_auc'] = cross_val_score(model, x, y, cv=5, scoring='average_precision').mean() # verbose=2

    return df_metrics

In [68]:
stocks = ['AAPL', 'AMZN', 'META', 'TSLA', 'SPC', 'NFLX', 'TWTR', 'GOOG']
forex = ['USD/JPY', 'EUR/GBP', 'USD/RUB', 'EUR/JPY', 'EUR/USD', 'GBP/JPY', 'GBP/USD', 'XAG/USD', 'XAU/USD']

In [71]:
date_from = '2020-01-01' #'2022-09-01'
date_to = date.today() #+ pd.to_timedelta(1, unit='d')
source = 'file'

if source == 'db':
    req = f"""
    select * from order_statistics where executed_date_time between '{date_from}' and '{date_to}'
    and account_clearing_code = 'live'
    order by executed_date_time"""

    engine = connect()
    orders = pd.DataFrame(engine.execute(req)).drop_duplicates()\
        [['account_id', 'order_strategy', 'order_symbol', 'price', 'quantity', 'quote_currency',
        'executed_date', 'executed_date_time', 'order_pl', 'position_code', 'position_effect', 'order_id', 'user_created_date_time']]\
                                                                .replace('USDT', 'USD')\
                                                               .replace('', np.nan).fillna({'quote_currency': 'USD'})
elif source == 'file':
    orders = pd.read_csv(r"/Users/p.matchenkov/NotebookProjects/users/data/input/all_history.csv", low_memory=False)\
        [['user_id', 'account_id', 'order_strategy', 'order_symbol', 'price', 'quantity', 'quote_currency', 'executed_date', 
        'executed_date_time', 'order_pl', 'position_code', 'position_effect', 'order_id', 'user_created_date_time']]\
                                                                .replace('USDT', 'USD')\
                                                               .replace('', np.nan).fillna({'quote_currency': 'USD'}).drop_duplicates()                                                             

orders = orders.loc[~orders['order_symbol'].isin(forex+stocks)]                                                       
orders['date'] = pd.to_datetime(orders['executed_date']) #- pd.to_timedelta(1, unit='d')
dates = tuple(orders['executed_date'].astype(str).unique())

quote_currencies_prices = get_price_from_db(dates)

orders = orders.merge(quote_currencies_prices, how='left', on=['executed_date', 'quote_currency'])\
              .replace('', np.nan).fillna(1)
orders['pnl'] = orders['order_pl'] * orders['close']
orders['volume'] = orders['price'] * orders['quantity'] * orders['close']
orders = orders.merge(comission_df, how='left', on='order_symbol')
orders['comission'] = orders['volume'] * orders['comission_rate']

In [72]:
result = orders.copy()


# getting leverages df to add to result 
accounts = result['account_id'].unique()
levereges = get_account_leverage(accounts=accounts)
levereges = levereges.merge(orders.groupby('account_id').agg({'comission': 'count'}), how='left', on='account_id')
levereges['leverage'] = levereges['name'].str.extract('\:(.*)')
levereges['leverage'] = levereges['leverage'].astype(float) * levereges['comission'].astype(float)
levereges['user_id'] = levereges['account_id'].str.extract('_(.*)_')


# sum features by week
result = result.groupby('user_id')\
    .agg({'pnl': sum, 'comission': 'sum', 'volume': ['sum', 'mean', 'median'],
        'order_pl': 'count', 'account_id': pd.Series.nunique, 'user_created_date_time': 'first'})\
    .reset_index()
result.columns = ['user_id', 'pnl', 'comission', 'volume_sum', 'volume_mean', 'volume_median', 'trades_qty', 'accounts_count', 'created_date']
result['created_date'] = pd.to_datetime(result['created_date']).dt.date#.astype(str)
result['account_age'] = (date.today() -  result['created_date']).dt.days
result['created_date'] = result['created_date'].astype(str)


# adding deposits
users_id = result['user_id'].unique()
result = result.merge(get_deposit(users_id), how='left', on='user_id')
result['deposit'] = result['deposit'].fillna(result['deposit'].median())
result['withdrawal'] = result['withdrawal'].fillna(result['withdrawal'].median())


# creating df with avg trade time
result = result.merge(avg_trade_time(orders), how='left', on='user_id')
result['trade_time'] = result['trade_time'].fillna(result['trade_time'].median())


# create df with counts of accs levereges and addint to result dataframe
levereges = levereges.groupby('user_id').agg({'leverage': 'sum'}).reset_index()


# adding first deposit amount in usd
result = result.merge(get_first_deposit(result['user_id']) , how='left', on='user_id')
result['first_dep_usd'] = result['first_dep_usd'].fillna(result['first_dep_usd'].median())


# add btc for created account_date
btc_price = yf.Ticker('BTC-USD')
btc_hist = btc_price.history(period="42mo").reset_index().drop_duplicates()
btc_hist['created_date'] = btc_hist['Date'].dt.date.astype(str)
btc_hist = btc_hist[['created_date', 'Close']]
result = result.merge(btc_hist, how='left', on='created_date')


In [73]:
result['mean_deposit'] = result['mean_deposit'].fillna(result['mean_deposit'].median())
result['mean_withdrawal'] = result['mean_withdrawal'].fillna(result['mean_withdrawal'].median())
result.isna().sum()

user_id              0
pnl                  0
comission            0
volume_sum           0
volume_mean          0
volume_median        0
trades_qty           0
accounts_count       0
created_date         0
account_age          0
deposit              0
mean_deposit         0
withdrawal           0
mean_withdrawal      0
trade_time           0
first_dep_usd        0
Close              501
dtype: int64

In [74]:
result.loc[result['created_date'] < '2019-03-01', 'Close'] = result['Close'].fillna(3700)
result.loc[(result['created_date'] >= '2019-03-01') & (result['created_date'] < '2019-06-01'), 'Close'] = result['Close'].fillna(4200)
result.loc[(result['created_date'] >= '2019-06-01') & (result['created_date'] < '2019-12-31'), 'Close'] = result['Close'].fillna(7000)

In [75]:
result['dep-withd'] = result['deposit'] - result['withdrawal']
result['dep+withd'] = result['deposit'] + result['withdrawal']
result['dep*withd'] = result['deposit'] * result['withdrawal']
result['dep/withd'] = result['deposit'] / result['withdrawal']
result['trades/accounts'] = result['trades_qty'] / result['accounts_count']
result['deposit/volume_sum'] = result['deposit'] / result['volume_sum']
result['deposit/trades_qty'] = result['deposit'] / result['trades_qty']
result['dep/btc'] = result['deposit'] / result['Close']

result['dep/withd'] = result['dep/withd'].fillna(0)

#result.head()

In [76]:
# coding target
result.loc[result['pnl']>=0, 'pnl'] = 0
result.loc[result['pnl']<0, 'pnl'] = 1
#result.tail()

In [ ]:
result.describe()

In [78]:
result.shape

(3154, 25)

## TRAIN

In [79]:
random = 42

x = result.drop(columns={'pnl', 'user_id', 'created_date'})
y = result['pnl']

scaler = StandardScaler()
x = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=random, test_size=0.25)

In [80]:
len(result.loc[result['pnl'] == 1]) / len(result)

0.8205453392517438

# Logical regression

In [81]:
params = {
     'penalty': ['l1', 'l2'],
     'solver': ['liblinear', 'saga']  
 }

log_reg = GridSearchCV(LogisticRegression(random_state=random, max_iter=10000), param_grid=params, cv=5)
log_reg.fit(x_train, y_train)

y_train_pred = log_reg.predict(x_train)
y_test_pred = log_reg.predict(x_test)

metrics_df = get_metrics(y_train,
                          y_train_pred,
                          y_test,
                          y_test_pred,
                          'LogisticRegression',
                          log_reg,
                          x,
                          y)
metrics_df

,model,train_accuracy,test_accuracy,recall,precision,F1,cv_roc_auc,pr_auc
0,LogisticRegression,0.83,0.81,0.99,0.82,0.90,0.62,0.88


# Desicion Tree

In [82]:
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(2, 100, 2),
    'max_features': ['auto', 'sqrt', 'log2']
}

dec_tree = GridSearchCV(DecisionTreeClassifier(random_state=random), param_grid=params, cv=5)
dec_tree.fit(x_train, y_train)

y_train_pred = dec_tree.predict(x_train)
y_test_pred = dec_tree.predict(x_test)

metrics_df = metrics_df.append(get_metrics(y_train,
                                           y_train_pred,
                                           y_test,
                                           y_test_pred,
                                           'DecisionTreeClassifier',
                                           model=dec_tree,
                                           x=x,
                                           y=y
                                           ))
metrics_df


,model,train_accuracy,test_accuracy,recall,precision,F1,cv_roc_auc,pr_auc
0,LogisticRegression,0.83,0.81,0.99,0.82,0.90,0.62,0.88
0,DecisionTreeClassifier,0.86,0.82,0.97,0.83,0.90,0.75,0.91


# GradientBoostingClassifier

In [83]:
params = {
     'criterion': ['friedman_mse', 'squared_error'], 
     'max_features': ['auto', 'sqrt', 'log2']
}
#      'learning_rate': np.arange(0.1 , 1.1, 0.1),      'min_weight_fraction_leaf': np.arange(0.0, 0.5, 0.1),
grad_boost_class = GridSearchCV(GradientBoostingClassifier(random_state=random), param_grid=params, cv=5)
grad_boost_class.fit(x_train, y_train)

y_train_pred = grad_boost_class.predict(x_train)
y_test_pred = grad_boost_class.predict(x_test)

metrics_df = metrics_df.append(get_metrics(y_train,
                                           y_train_pred,
                                           y_test,
                                           y_test_pred,
                                           'GradientBoostingClassifier',
                                           grad_boost_class,
                                           x,
                                           y))
metrics_df

,model,train_accuracy,test_accuracy,recall,precision,F1,cv_roc_auc,pr_auc
0,LogisticRegression,0.83,0.81,0.99,0.82,0.90,0.62,0.88
0,DecisionTreeClassifier,0.86,0.82,0.97,0.83,0.90,0.75,0.91
0,GradientBoostingClassifier,0.90,0.86,0.98,0.87,0.92,0.81,0.94


# Random forest

In [84]:
params = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2', None] 
} 

rand_forest = GridSearchCV(RandomForestClassifier(random_state=random), param_grid=params)
rand_forest.fit(x_train, y_train)

y_train_pred = rand_forest.predict(x_train)
y_test_pred = rand_forest.predict(x_test)

metrics_df = metrics_df.append(get_metrics(y_train,
                                           y_train_pred,
                                           y_test,
                                           y_test_pred,
                                           'RandomForestClassifier',
                                           rand_forest,
                                           x,
                                           y))
metrics_df

,model,train_accuracy,test_accuracy,recall,precision,F1,cv_roc_auc,pr_auc
0,LogisticRegression,0.83,0.81,0.99,0.82,0.90,0.62,0.88
0,DecisionTreeClassifier,0.86,0.82,0.97,0.83,0.90,0.75,0.91
0,GradientBoostingClassifier,0.90,0.86,0.98,0.87,0.92,0.81,0.94
0,RandomForestClassifier,1.00,0.87,0.97,0.88,0.93,0.83,0.94


# SVM

In [85]:
svc = SVC(probability=True, random_state=random, kernel='rbf')
svc.fit(x_train, y_train)

y_train_pred = svc.predict(x_train)
y_test_pred = svc.predict(x_test)

metrics_df = metrics_df.append(get_metrics(y_train,
                                           y_train_pred,
                                           y_test,
                                           y_test_pred,
                                           'SVM',
                                           svc,
                                           x,
                                           y))
metrics_df

,model,train_accuracy,test_accuracy,recall,precision,F1,cv_roc_auc,pr_auc
0,LogisticRegression,0.83,0.81,0.99,0.82,0.90,0.62,0.88
0,DecisionTreeClassifier,0.86,0.82,0.97,0.83,0.90,0.75,0.91
0,GradientBoostingClassifier,0.90,0.86,0.98,0.87,0.92,0.81,0.94
0,RandomForestClassifier,1.00,0.87,0.97,0.88,0.93,0.83,0.94
0,SVM,0.83,0.81,1.00,0.81,0.90,0.67,0.90
